# Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install packages

In [ ]:
pip install assemblyai

In [ ]:
pip install praat-textgrids

In [ ]:
pip install pydub

In [ ]:
pip install SpeechRecognition

# Functions

In [ ]:
# transcribe wav files and save transcriptions in a dictionary
def transcribe(api_key, filenames, wav_dir, output_dictionary_dir, language_code):
    import os
    import assemblyai as aai
    import pickle

    # your AssemblyAI API key
    aai.settings.api_key = api_key

    # set language
    config = aai.TranscriptionConfig(language_code=language_code)

    # create transcriber
    transcriber = aai.Transcriber(config=config)

    # initialize transcription dictionary
    transc_dict = {}

    for file in filenames:
        # load wav file
        filepath = os.path.join(wav_dir, file)

        # transcribe wav file
        transcript = transcriber.transcribe(filepath)

        if transcript.error:
            print(file, transcript.error)
            transc_dict[file] = 'not_recognized'
        else:
            # save transcription in dictionary
            transc_dict[file] = transcript.text  or 'not_recognized'
            print(f"{file}: {transcript.text}")

    # save the dictionary
    with open(os.path.join(output_dictionary_dir, "transcriptions.p"), "wb") as f:
        pickle.dump(transc_dict, f)

    # format final transcriptions
    final_transc = {os.path.splitext(key)[0]: val for key, val in transc_dict.items()}

    return final_transc

In [ ]:
# This function is adapted from: https://github.com/Masato-Nakamura-3/TransCloze
def chronset_results_to_dictionary(chronset_dir, output_onset_dir):
    import os
    import pandas as pd
    import pickle

    onset_dict = dict()
    for textf in [t for t in os.listdir(chronset_dir) if ".txt" in t]:
        onset_df = pd.read_table(os.path.join(chronset_dir, textf), names = ["wav", "rt"])
        temp_dict = dict(zip([os.path.splitext(w)[0] for w in onset_df["wav"]], onset_df["rt"] / 1000))
        onset_dict.update(temp_dict)

    # save the dictionary
    with open(os.path.join(output_onset_dir, "onset.p"), "wb") as f:
        pickle.dump(onset_dict, f)

    return onset_dict

In [ ]:
# This function is adapted from: https://github.com/Masato-Nakamura-3/TransCloze
# This function generates .TextGrid files for Praat
# You can optionally pass a dictionary for the onset and/or for the duration of the files
# The onset function is basically for Chronset
# You can also give it context info to help hand correction

# If the audio files have heterogeneous durations, add an argument of "inconsistent = 1"

# The function replaces the RT with 0.1 if the RT is 0 or shorter, or is NaN, producing an error message

def create_textgrids(dict_transcription, output_dir, wav_dir, onset = dict(), inconsistent = False):
    import os
    import textgrids
    import numpy as np
    from pydub import AudioSegment

    filenames = [os.path.splitext(i)[0] for i in os.listdir(wav_dir) if (".wav" in i)]

    for k in filenames:
        # Load the TextGrid template and edit it
        tg = textgrids.TextGrid("/content/drive/MyDrive/FastTranscribeAlign/tg_raw/temp_note.TextGrid")

        # If the duration of the audio files are inconsistent, edit each TextGrid files to match the duration of the audio files
        if inconsistent == True:
            audio = AudioSegment.from_file(os.path.join(wav_dir, k + ".wav"))
            tg["words"][1].xmax = tg["words"][-1].xmin = audio.duration_seconds - 0.001

            if tg.xmax != audio.duration_seconds:
                tg.xmax = audio.duration_seconds

                for tier in tg.keys():
                    tg[tier][-1].xmax = audio.duration_seconds

        tg["words"][1].text = dict_transcription[k + ".wav"]

        if k in onset.keys():
            # If the onset is not detected or is negative, set 0.1
            if np.isnan(onset[k]) or onset[k] <= 0 :
                print("No onset for ", k)
                tg["words"][0].xmax = tg["words"][1].xmin = 0.1
            else:
                tg["words"][0].xmax = tg["words"][1].xmin = onset[k]

        item_id = k.split("_")[1]

        tg.write(os.path.join(output_dir, k + ".TextGrid"))

    print("All textgrid files generated.")

# Designate paths and run functions

In [ ]:
# This pipeline uses automatic speech-to-text software provided by https://www.assemblyai.com/
# Create an account and replace the code below with your personal API key.
api_key = 'your_api_key'

In [ ]:
# path to folder containing wav files
wav_dir = '/content/drive/MyDrive/FastTranscribeAlign/wav'

# get list of wav file names
import os
filenames = [w for w in os.listdir(wav_dir) if (".wav" in w)]

# path to output folder with transcription dictionary
output_dictionary_dir = '/content/drive/MyDrive/FastTranscribeAlign/transc_output'

# path to zip folder with wav files for chronset
chronset_input_dir = '/content/drive/MyDrive/FastTranscribeAlign/chronset_input'

# path to output folder with onset
chronset_dir = '/content/drive/MyDrive/FastTranscribeAlign/chronset_results'

# path to output folder with onset
output_onset_dir = '/content/drive/MyDrive/FastTranscribeAlign/onset_output'

# path to output folder with textgrids (with transcriptions)
tg_dir = '/content/drive/MyDrive/FastTranscribeAlign/tg_raw'

In [ ]:
# generate transcriptions and store them in a dictionary
# replace "language_code" with your language code (e.g., American English : "en_us"); check language_code here: https://www.assemblyai.com/docs/getting-started/supported-languages
language_code = "your_language_code"
dict_transcription = transcribe(api_key, filenames, wav_dir, output_dictionary_dir, language_code)

# Run script below if you only need transcriptions + speech onset data (get onset data using Chronset)

This section uses Chronset to get onset data

In [ ]:
import os
import zipfile
from pydub import AudioSegment
import speech_recognition as sr

# Set the name of the zip files
# The output files will be named as "{experiment_name}_{number}".zip (e.g. planet_cloze_1.zip)
zipname = "experiment_name_"

# # Check wav files for errors
# for f in filenames:
#     fsize = os.path.getsize(os.path.join(wav_dir, f))
#     if fsize < 100000:
#         print(f)
#         print(fsize)
#         with sr.AudioFile(os.path.join(wav_dir, f)) as wavf:
#             print(wavf.DURATION)

# Create batches of 500 wav files (i.e., maximum file counts Chronset can handle)
# Make lists of 500 or fewer wav files
fewerthan500 = []
temp = []
for f in filenames:
    temp.append(f)
    if len(temp) == 500:
        fewerthan500.append(temp)
        temp = []
if len(temp) > 0:
    fewerthan500.append(temp)

# Total nuber of wav files
len(filenames)
# Number of zip files
len(fewerthan500)

# Output zip files (with 500 or fewer wav files)
fnum = 0
for filelist in fewerthan500:
    fnum = fnum + 1
    with zipfile.ZipFile(os.path.join(chronset_input_dir, zipname+str(fnum) + ".zip"), 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
        for f in filelist:
            new_zip.write(os.path.join(wav_dir, f), arcname = f)

Go to https://www.bcbl.eu/databases/chronset and upload each zip file that is in the **chronset_input** folder.

Retrieve Chronset results from your mailbox and upload those files to the **chronset_results** folder.

In [ ]:
# get onsets and store them in a dictionary
onset = chronset_results_to_dictionary(chronset_dir, output_onset_dir)
onset = pickle.load( open( "/content/drive/MyDrive/FastTranscribeAlign/onset.p", "rb" ) )

In [ ]:
# generate textgrid files and store them in tg_dir_final
import pickle
transcriptions = pickle.load( open( "/content/drive/MyDrive/FastTranscribeAlign/transcriptions.p", "rb" ) )
create_textgrids(transcriptions, tg_dir, wav_dir, onset, inconsistent = True)

# Run script below if you need transcriptions + onset and alignment of multi-word utterances

This script does not get timing data - follow next steps in the pipeline on Github to get timing data using MFA

In [ ]:
# generate textgrid files and store them in tg_dir_final
import pickle
transcriptions = pickle.load( open( "/content/drive/MyDrive/FastTranscribeAlign/transcriptions.p", "rb" ) )
create_textgrids(transcriptions, tg_dir, wav_dir, inconsistent = True)